In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all' # to print multiple outputs from the same cell
import math
import utils
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns
import itertools
from operator import index
from collections import defaultdict
from scipy.stats import pearsonr
from datetime import datetime
#from lingua import Language, LanguageDetectorBuilder

Creates the combined dataset of the cleaned and generalized user and tweet datasets. The file that is created is the merged_dataset_cleaned

In [2]:
users_df = pd.read_csv("dataset/users_dataset_cleaned.csv")
tweets_df = pd.read_csv("dataset/tweets_dataset_cleaned.csv", lineterminator='\n')

In [3]:
merged_df = pd.merge(users_df, tweets_df, left_on=["user_id"], right_on=["user_id"])

removing tweets made before author account creation date

In [4]:
# Removes any duplicate columns as a result of the merge
#merged_df = merged_df.loc[:,~merged_df.columns.duplicated()]#.copy()  <-- Add this if we have to avoid memory issues down the line
merged_df.rename(columns={"created_at_x" : "account_created"}, inplace=True)
merged_df.rename(columns={"created_at_y" : "tweet_created"}, inplace=True)
rows_to_delete = merged_df[merged_df["account_created"] > merged_df["tweet_created"]].index
merged_df.drop(rows_to_delete, inplace=True)

In [5]:
merged_df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10460637 entries, 0 to 10460636
Data columns (total 15 columns):
 #   Column           Dtype 
---  ------           ----- 
 0   user_id          int64 
 1   name             object
 2   lang             object
 3   bot              int64 
 4   account_created  object
 5   statuses_count   int64 
 6   tweet_id         int64 
 7   retweet_count    int64 
 8   reply_count      int64 
 9   favorite_count   int64 
 10  num_hashtags     int64 
 11  num_urls         int64 
 12  num_mentions     int64 
 13  tweet_created    object
 14  text             object
dtypes: int64(10), object(5)
memory usage: 1.2+ GB


In [ ]:
merged_df.to_csv("./dataset/merged_dataset_cleaned.csv",index=False) # Removes the counting of the Index rows